In [1]:
import pandas as pd

df = pd.read_csv("deanonymous_score_100.csv")

df["confidence"] = 0.5
df["trustiness"] = 0.5
df["reliability"] = 0.7

max_iterations = 10000
epsilon = 0.01

for iteration in range(max_iterations):
    df["trustiness_prev"] = df["trustiness"]
    df["reliability_prev"] = df["reliability"]
    df["confidence_prev"] = df["confidence"]
    df["weighted_score"] = df["deanonymous"] * df["confidence"]
    
    trustiness_updates = df.groupby("to")["weighted_score"].mean()
    df["trustiness"] = df["to"].map(trustiness_updates)

    reliability_updates = df.groupby("from")["confidence"].mean()
    df["reliability"] = df["from"].map(reliability_updates)

    df["trustiness"] = df["trustiness"].clip(0, 1)
    df["reliability"] = df["reliability"].clip(0, 1)

    df["confidence"] = (df["reliability"] + (1 - abs(df["deanonymous"] - df["trustiness"]))) / 2

    df["confidence"] = df["confidence"].clip(0, 1)

    delta_t = (df["trustiness"] - df["trustiness_prev"]).abs().sum()
    delta_r = (df["reliability"] - df["reliability_prev"]).abs().sum()
    delta_c = (df["confidence"] - df["confidence_prev"]).abs().sum()
    delta = max(delta_t, delta_r, delta_c)

    print(f"Iteration {iteration + 1}: ΔT = {delta_t:.4f}, ΔR = {delta_r:.4f}, ΔC = {delta_c:.4f}, Δ = {delta:.4f}")

    if delta < epsilon:
        print("Convergence achieved!")
        break

df["risk"] = (1 - df["reliability"]) * 10

Iteration 1: ΔT = 948619.3534, ΔR = 827292.8000, ΔC = 464893.8198, Δ = 948619.3534
Iteration 2: ΔT = 65690.2256, ΔR = 454973.1258, ΔC = 252311.6748, Δ = 454973.1258
Iteration 3: ΔT = 62724.6059, ΔR = 252099.7458, ΔC = 151497.0182, Δ = 252099.7458
Iteration 4: ΔT = 50088.6738, ΔR = 151255.5274, ΔC = 96962.3819, Δ = 151255.5274
Iteration 5: ΔT = 39688.8419, ΔR = 96804.5121, ΔC = 66028.7243, Δ = 96804.5121
Iteration 6: ΔT = 32024.2336, ΔR = 65928.9349, ΔC = 47769.7701, Δ = 65928.9349
Iteration 7: ΔT = 26484.6741, ΔR = 47693.5781, ΔC = 36503.6914, Δ = 47693.5781
Iteration 8: ΔT = 22401.6026, ΔR = 36451.4561, ΔC = 29160.6975, Δ = 36451.4561
Iteration 9: ΔT = 19327.0517, ΔR = 29124.1461, ΔC = 24111.1606, Δ = 29124.1461
Iteration 10: ΔT = 16961.8474, ΔR = 24080.7261, ΔC = 20462.2995, Δ = 24080.7261
Iteration 11: ΔT = 15098.0790, ΔR = 20437.3864, ΔC = 17733.3846, Δ = 20437.3864
Iteration 12: ΔT = 13595.3793, ΔR = 17712.2388, ΔC = 15631.0887, Δ = 17712.2388
Iteration 13: ΔT = 12359.0826, ΔR = 1

In [ ]:
risk_df = df.groupby("from")["risk"].mean().reset_index()

addresses = pd.read_csv("labeled_addresses.csv")
addresses = addresses[addresses["label"] == 1]["Address"]

merged_df = pd.merge(
    addresses.to_frame(name="address"),
    risk_df.rename(columns={"from": "address"}),
    on="address",
    how="left"
)

In [6]:
len(merged_df[merged_df['risk'] < 6])

126

In [7]:
len(merged_df[merged_df['risk'] >= 6])

109

In [8]:
len(merged_df[merged_df['risk'].isna()])

8

In [ ]:
risk_df = df.groupby("from")["risk"].mean().reset_index()

addresses = pd.read_csv("labeled_addresses.csv")
addresses = addresses[addresses["label"] == 0]["Address"]

merged_df = pd.merge(
    addresses.to_frame(name="address"),
    risk_df.rename(columns={"from": "address"}),
    on="address",
    how="left"
)

In [10]:
len(merged_df[merged_df['risk'] < 6])

516

In [11]:
len(merged_df[merged_df['risk'] >= 6])

20

In [13]:
len(merged_df[merged_df['risk'].isna()])

24